# Generate core-hole pseudopotentials 

For the source code, see [core_hole_pseudo](https://workgraph-collections.readthedocs.io/en/latest/ase/espresso/module.html#workgraph_collections.ase.espresso.core_hole_pseudo.generate_core_hole_pseudo).


## Introduction
The core correction can be expressed as:

$$
\Delta E = (E_{\text{tot}}^{\text{excited}} - E_{\text{totps}}^{\text{excited}}) - (E_{\text{tot}}^{\text{ground}} - E_{\text{totps}}^{\text{ground}})
$$

Where:

- $E_{\text{totps}}$ is the energy of the pseudo-potential part of the system.
- $E_{\text{tot}}$ is the energy of the whole system.

## Visualizing the WorkGraph Builder


In [5]:
from workgraph_collections.ase.espresso.core_hole_pseudo import core_hole_pseudo_workgraph

task = core_hole_pseudo_workgraph.task()
task.to_html()
# comment out this line to visualize the workgraph in the Jupyter notebook
# task

## Visualizing the WorkGraph

In [1]:
from workgraph_collections.ase.espresso.core_hole_pseudo import core_hole_pseudo_workgraph

wg = core_hole_pseudo_workgraph()
wg.to_html()
# comment out this line to visualize the workgraph in the Jupyter notebook
# wg

## Example: Core-hole pseudopotential for Pt

### Prepare the inputs and submit the workflow


In [1]:
from aiida import load_profile
from workgraph_collections.ase.espresso.core_hole_pseudo import core_hole_pseudo_workgraph
from copy import deepcopy

load_profile()

metadata = {
    "options": {
        'prepend_text' : """eval "$(conda shell.posix hook)"
        conda activate aiida
        export OMP_NUM_THREADS=1
        """,
    }
}

input_data = {
    "input": {
        "title": "Pt",
        "zed": 78.0,
        "rel": 1,
        "config": "[Xe] 4f14 6s1 6p0 5d9",
        "iswitch": 3,
        "dft": "PBE"
    },
    "inputp": {
        "lpaw": True,
        "use_xsd": False,
        "pseudotype": 3,
        "file_pseudopw": "Pt.pbe-n-kjpaw_psl.1.0.0.UPF",
        "author": "AiiDA",
        "lloc": -1,
        "rcloc": 2.4,
        "which_augfun": "PSQ",
        "rmatch_augfun_nc": True,
        "nlcc": True,
        "new_core_ps": True,
        "rcore": 1.8,
        "tm": True
    },
}
pseudo_potential_test_cards = """
6
6S  1  0  1.00  0.00  2.00  2.20  0.0
6S  1  0  0.00  4.40  2.00  2.20  0.0
6P  2  1  0.00  0.00  2.30  2.50  0.0
6P  2  1  0.00  6.40  2.30  2.50  0.0
5D  3  2  9.00  0.00  1.00  2.20  0.0
5D  3  2  0.00  0.80  1.00  2.20  0.0
"""
ground_inputs = {"input_data": input_data,
                 "pseudo_potential_test_cards": pseudo_potential_test_cards,
                 "metadata": metadata,
                 "computer": "localhost"
                 }
input_data_core_hole = deepcopy(input_data)
# remove one electron from 4f
input_data_core_hole["input"]["config"] = "[Xe] 4f13 6s1 6p0 5d9"
input_data_core_hole["inputp"]["file_pseudopw"] = "Pt.star4f.pbe-n-kjpaw_psl.1.0.0.UPF"
core_hole_inputs = {"input_data": input_data_core_hole,
                    "pseudo_potential_test_cards": pseudo_potential_test_cards,
                    "computer": "localhost",
                    "metadata": metadata
                    }

#------------------------- Set the inputs -------------------------
wg = core_hole_pseudo_workgraph(ground_inputs,
                                core_hole_inputs)
#------------------------- Submit the calculation -------------------
# wg.run()
wg.submit(wait=True, timeout=200)
#------------------------- Print the output -------------------------
print('Correction:                  {:0.3f} eV'.format(wg.tasks['calc_correction'].outputs["result"].value.value))



WorkGraph process created, PK: 35419
Correction:                  207.613 eV
